In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from typing import Iterable
import libs.padding as P

import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils import data

dev = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(dev)
print(device)

cpu


In [2]:
bsz, channels, depth, height, width = 4, 3, 24, 24, 24
input_size = (bsz, channels, depth, height, width)

In [3]:
_kernel_size = torch.Tensor((3, 3, 3)).int()
_stride = torch.Tensor((1, 2, 3)).int()
_dilation = torch.Tensor((0, 1, 2)).int()
_padding = torch.Tensor((1, 1, 1)).int()

X = torch.rand(input_size)
print(X.shape)

pad = P.Pad3D(padding=_padding)
Xpad = pad(X)
print(Xpad.shape)

torch.Size([4, 3, 24, 24, 24])
torch.Size([4, 3, 26, 26, 26])


In [8]:
class Unfold3d(nn.Module):
    def __init__(self, kernel_size, stride, padding, dilation):
        super().__init__()
        self.pad = P.Pad3D(padding)
        self.kernel_size = torch.Tensor(kernel_size).flatten().int()
        self.stride = torch.Tensor(stride).flatten().int()
        self.dilation = torch.Tensor(dilation).flatten().int()

    def _dstarts(self, _d):
        return torch.arange(0, _d-self.kernel_size[0], self.stride[0])

    def _hstarts(self, _h):
        return torch.arange(0, _h-self.kernel_size[1], self.stride[1])

    def _wstarts(self, _w):
        return torch.arange(0, _w-self.kernel_size[2], self.stride[2])
    
    def forward(self, X):
        out = self.pad(X)
        _, _, _d, _h, _w = out.shape
        windows = torch.stack(torch.meshgrid(self._dstarts(_d), self._hstarts(_h), self._wstarts(_w), indexing='ij'), dim=3).reshape((-1, 3))

In [16]:
_, _, _d, _h, _w = Xpad.shape
_midpoints = (_kernel_size - 1).int()

dstarts = torch.arange(0, _d-_kernel_size[0], _stride[0])
dends = dstarts + _kernel_size[0] + _dilation[0] * _midpoints[0]

hstarts = torch.arange(0, _h-_kernel_size[1], _stride[1])
hends = hstarts + _kernel_size[1] + _dilation[1] * _midpoints[1]

wstarts = torch.arange(0, _w-_kernel_size[2], _stride[2])
wends = wstarts + _kernel_size[2] + _dilation[2] * _midpoints[2]

window_starts = torch.stack(torch.meshgrid(dstarts, hstarts, wstarts, indexing='ij'), dim=3).reshape((-1, 3))
window_ends = torch.stack(torch.meshgrid(dends, hends, wends, indexing='ij'), dim=3).reshape((-1, 3))
window_mult = torch.zeros((window_ends[0][0]-window_starts[0][0], window_ends[0][1]-window_starts[0][1], window_ends[0][2]-window_starts[0][2]))

for i in range(window_starts.shape[0]):
    start, end = window_starts[i], window_ends[i]
    print(start, end)
    break

tensor([0, 0, 0]) tensor([3, 5, 7])


tensor([[1.]])
tensor([[1.]])
